In [ ]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 29.1 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import nltk
import collections
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, layers, losses, Model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from scipy import spatial

## BERT

In [ ]:
!pip install spacy-sentence-bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 5.5 MB 55.8 MB/s 
     |████████████████████████████████| 1.3 MB 51.7 MB/s 
     |████████████████████████████████| 163 kB 67.3 MB/s 
     |████████████████████████████████| 7.6 MB 55.8 MB/s 
  Created wheel for spacy-sentence-bert: filename=spacy_sentence_bert-0.1.2-py3-none-any.whl size=27201 sha256=592b6723818afe03ac66e00bc178307dbc0fb99a5494b17a27b33e5c692a028b
  Stored in directory: /root/.cache/pip/wheels/54/e2/4a/e3f24fb6729ada3c83377de5217261c0e2f153c5a6f36d37d0
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=081d2b2157e598596fcac28b44a49a92605908941282b1cae31b8c0b96ae36b3
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built spacy-sentence-bert sen

In [ ]:
import spacy_sentence_bert
import spacy

In [ ]:
nlp = spacy_sentence_bert.load_model('en_stsb_roberta_large')

Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv").iloc[:200,:]

X = np.array([nlp(df.iloc[i,0]).vector for i in range(df.shape[0])])

y = np.asarray(df.iloc[:,1])
y[y=="positive"] = 1
y[y=="negative"] = 0
y = y.reshape(-1,1).astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train.shape

(100, 1024)

In [ ]:
def encoder_fn(input_shape):

    input = layers.Input((len_bag,))
    x = tf.keras.layers.BatchNormalization()(input)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    outputlayer = layers.Dense(64, activation='relu')(x)

    embedding_network = tf.keras.Model(input, outputlayer)
 
    model = Model(inputs=input, outputs=outputlayer)
    return model

def add_projection_head(input_shape, encoder):
    inputs = layers.Input(shape=input_shape, name='inputs')
    features = encoder(inputs)
    outputs = layers.Dense(64, activation='relu', 
                      name='projection_head', 
                      dtype='float32')(features)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

  

In [ ]:
len_bag = X_train.shape[1]
model1 = encoder_fn(len_bag)

In [ ]:
model1.summary()

Model: "model_162"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_51 (InputLayer)       [(None, 1024)]            0         
                                                                 
 batch_normalization_158 (Ba  (None, 1024)             4096      
 tchNormalization)                                               
                                                                 
 dense_237 (Dense)           (None, 512)               524800    
                                                                 
 dense_238 (Dense)           (None, 128)               65664     
                                                                 
 batch_normalization_159 (Ba  (None, 128)              512       
 tchNormalization)                                               
                                                                 
 dense_239 (Dense)           (None, 64)                82

In [ ]:
def classifier_fn(input_shape, encoder, trainable=False):
    for layer in encoder.layers:
        layer.trainable = trainable
        
    inputs = layers.Input(shape=input_shape, name='inputs')
    
    features = encoder(inputs)
    features = layers.Dense(64, activation='relu')(features)
    features = tf.keras.layers.BatchNormalization()(features)
    features = layers.Dense(128, activation='relu')(features)
    features = layers.Dense(512, activation='relu')(features)
    features = layers.Dense(64, activation='relu')(features)
    features = tf.keras.layers.BatchNormalization()(features)
    outputs = layers.Dense(2, activation='softmax', 
                      name='outputs', dtype='float32')(features)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
class SupervisedContrastiveLoss(losses.Loss):
    def __init__(self, temperature=0.1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, ft_vectors, sample_weight=None):
        # Normalize feature vectors
        ft_vec_normalized = tf.math.l2_normalize(ft_vectors, axis=1)
        print(ft_vec_normalized.shape)
        # Compute logits
        logits = tf.divide(
            tf.matmul(ft_vec_normalized, 
                      tf.transpose(ft_vec_normalized)
            ), self.temperature
        )
        return tfa.losses.contrastive_loss(tf.squeeze(labels), logits)

In [ ]:
encoder = encoder_fn(len_bag) # Get the encoder
encoder_proj = add_projection_head(len_bag,encoder)
# Add the projection head to the encoder
encoder_proj.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
loss=SupervisedContrastiveLoss(temperature=0.2))

In [ ]:
encoder_proj.fit(x=X_train, y=y_train,
          validation_data=(X_test, y_test), 
          batch_size=32,
          epochs=10)

Epoch 1/10
(None, 64)
(None, 64)
4/4 [==============================] - 1s 76ms/step - loss: 2.2617 - val_loss: 4.8534
Epoch 2/10
4/4 [==============================] - 0s 11ms/step - loss: 1.3581 - val_loss: 4.1061
Epoch 3/10
4/4 [==============================] - 0s 11ms/step - loss: 1.1906 - val_loss: 3.4616
Epoch 4/10
4/4 [==============================] - 0s 11ms/step - loss: 0.9659 - val_loss: 2.9656
Epoch 5/10
4/4 [==============================] - 0s 16ms/step - loss: 0.8722 - val_loss: 2.5881
Epoch 6/10
4/4 [==============================] - 0s 13ms/step - loss: 0.7592 - val_loss: 2.2862
Epoch 7/10
4/4 [==============================] - 0s 12ms/step - loss: 0.7269 - val_loss: 2.0863
Epoch 8/10
4/4 [==============================] - 0s 14ms/step - loss: 0.6617 - val_loss: 1.9390
Epoch 9/10
4/4 [==============================] - 0s 16ms/step - loss: 0.6549 - val_loss: 1.8257
Epoch 10/10
4/4 [==============================] - 0s 12ms/step - loss: 0.6399 - val_loss: 1.7219


In [ ]:
model = classifier_fn((len_bag,), 
                          encoder, # trained encoder
                          trainable=False) # with frozen weights

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                  loss=losses.SparseCategoricalCrossentropy(), 
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(x=X_train, y=y_train,
          validation_data=(X_test, y_test), 
          batch_size=32,
          epochs=25)

Epoch 1/25
4/4 [==============================] - 1s 81ms/step - loss: 0.7831 - categorical_accuracy: 0.4900 - val_loss: 0.7201 - val_categorical_accuracy: 0.0000e+00
Epoch 2/25
4/4 [==============================] - 0s 17ms/step - loss: 0.6917 - categorical_accuracy: 0.5400 - val_loss: 0.7158 - val_categorical_accuracy: 0.0000e+00
Epoch 3/25
4/4 [==============================] - 0s 17ms/step - loss: 0.5691 - categorical_accuracy: 0.5300 - val_loss: 0.7115 - val_categorical_accuracy: 0.0000e+00
Epoch 4/25
4/4 [==============================] - 0s 20ms/step - loss: 0.4629 - categorical_accuracy: 0.5300 - val_loss: 0.7072 - val_categorical_accuracy: 0.0100
Epoch 5/25
4/4 [==============================] - 0s 17ms/step - loss: 0.3969 - categorical_accuracy: 0.5200 - val_loss: 0.7025 - val_categorical_accuracy: 0.0500
Epoch 6/25
4/4 [==============================] - 0s 16ms/step - loss: 0.3823 - categorical_accuracy: 0.5400 - val_loss: 0.6976 - val_categorical_accuracy: 0.0800
Epoch 7/25

In [ ]:
y_test.sum()

41.0

In [ ]:
y_test_pred = model.predict(X_test)
results = pd.DataFrame(y_test_pred)
results["true"] = y_test
results.iloc[:50,:]

4/4 [==============================] - 0s 3ms/step


,0,1,true
0,0.393939,0.606061,1.0
1,0.643072,0.356928,0.0
2,0.534940,0.465060,1.0
3,0.540644,0.459356,0.0
4,0.482275,0.517725,1.0
5,0.510108,0.489892,1.0
6,0.558813,0.441187,0.0
7,0.677673,0.322327,0.0
8,0.484700,0.515300,0.0
9,0.442162,0.557838,1.0


## BERT - CV

In [ ]:
# Define the contrastive loss function, NT_Xent (Tensorflow version)
def NT_Xent_tf(zi, zj, tau=1):
    """ Calculates the contrastive loss of the input data using NT_Xent. The
    equation can be found in the paper: https://arxiv.org/pdf/2002.05709.pdf
    (This is the Tensorflow implementation of the standard numpy version found
    in the NT_Xent function).
    
    Args:
        zi: One half of the input data, shape = (batch_size, feature_1, feature_2, ..., feature_N)
        zj: Other half of the input data, must have the same shape as zi
        tau: Temperature parameter (a constant), default = 1.

    Returns:
        loss: The complete NT_Xent constrastive loss
    """
    z = tf.cast(tf.concat((zi, zj), 0), dtype=tf.float32)
    loss = 0
    for k in range(zi.shape[0]):
        # Numerator (compare i,j & j,i)
        i = k
        j = k + zi.shape[0]
        # Instantiate the cosine similarity loss function
        cosine_sim = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.NONE)
        sim = tf.squeeze(- cosine_sim(tf.reshape(z[i], (1, -1)), tf.reshape(z[j], (1, -1))))
        numerator = tf.math.exp(sim / tau)

        # Denominator (compare i & j to all samples apart from themselves)
        sim_ik = - cosine_sim(tf.reshape(z[i], (1, -1)), z[tf.range(z.shape[0]) != i])
        sim_jk = - cosine_sim(tf.reshape(z[j], (1, -1)), z[tf.range(z.shape[0]) != j])
        denominator_ik = tf.reduce_sum(tf.math.exp(sim_ik / tau))
        denominator_jk = tf.reduce_sum(tf.math.exp(sim_jk / tau))

        # Calculate individual and combined losses
        loss_ij = - tf.math.log(numerator / denominator_ik)
        loss_ji = - tf.math.log(numerator / denominator_jk)
        loss += loss_ij + loss_ji
    
    # Divide by the total number of samples
    loss /= z.shape[0]

    return loss



input = layers.Input((len_bag,))
x = layers.Dense(500)(input)
x = layers.Dense(50)(x)
x = tf.math.l2_normalize(x)

embedding_network = tf.keras.Model(input, x)


input_1 = layers.Input((len_bag,))
input_2 = layers.Input((len_bag,))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)


output_layer = ContrastiveLossLayer()(tower_1, tower_2)
siamese = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)

siamese.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=["accuracy"])
siamese.summary()

TypeError: ignored